In [ ]:
#  pip install --user git+https://github.com/LAL/trackml-library

In [ ]:
import torch
# def format_pytorch_version(version):
#   return version.split('+')[0]

# TORCH_version = torch.__version__
# TORCH = format_pytorch_version(TORCH_version)

# def format_cuda_version(version):
#   return 'cu' + version.replace('.', '')

# CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

# !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-geometric 

In [ ]:
#basic libraries
import numpy as np 
import pandas as pd
import trackml.dataset
import sys    
import os
import shutil
import time
#proccesing data 
from data_preprocess import *    # reads data from data_preprocess.py
#ML libraries
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from collections import namedtuple
from torch_geometric.data import Data

import warnings
warnings.simplefilter("ignore", UserWarning)


## Data Filters

In [ ]:
#######################ENRICH DATA#####################################
# Calculate derived hits variables
def initial_params(df):
  """ Calculate momentum radius, phi and ita"""
  df['pt'] = np.sqrt(df.tpx**2 + df.tpy**2)
  df['r'] = np.sqrt(df.x**2 + df.y**2)
  df['phi'] = np.arctan2(df.y, df.x)
  def calc_ita(r, z):
      theta = np.arctan2(r, z)
      return -1. * np.log(np.tan(theta / 2.)), theta
  df['ita'], df['theta'] = calc_ita(df['r'],df['z'])

  return df

def ordered_layer_id(df):
    """order layers from 0 to N"""
    df = df.sort_values(by=['volume_id','layer_id','z']).reset_index(drop = True) 
    new_layers_array = np.zeros(len(df))
    i = 0
    idx = 0
    for vol in df.volume_id.unique():
      temp_df = df.loc[df.volume_id == vol]
      for layer in temp_df.layer_id.unique():
        temp2_df = temp_df.loc[temp_df.layer_id == layer]
        new_layers_array[idx:len(temp2_df)+idx]=i
        i+=1
        idx+=len(temp2_df)
    df['new_layer_id'] = new_layers_array.astype(int)
    return df

In [ ]:
#######################SECTION FILTERS#####################################
# firts use the geomerty of the detector to reduce data, barrel, layers, and sectioning
def sections(df,phi_sec,ita_sec, overlap_phi = 0, overlap_ita = 0):
   
  #use only important columns
  df = df[['new_hit_id', 'new_particle_id', 'particle_id','weight','new_layer_id','layer_id','volume_id','x', 'y', 'z','pt','tpx','tpy', 'r', 'phi','ita','event_id']]
  phi_range = (-np.pi, np.pi)
  ita_range =  (-3, 3)
  n_phi_sections = phi_sec          # Towards a realistic track reconstruction algorithm based
  n_ita_sections = ita_sec          #on graph neural networks for the HL-LHC

  phi_edges = np.linspace(*phi_range, num=n_phi_sections+1)   #4 sections in range of phi 
  ita_edges = np.linspace(*ita_range, num=n_ita_sections+1)   #2 sections in range of ita
  def split_detector_sections(df, phi_edges, ita_edges):
      """Split hits according to provided phi and ita boundaries."""
      hits_sections = []
      # Loop over sections on phi
      for i in range(len(phi_edges) - 1):
          if np.round(abs(phi_edges[i] - overlap_phi),4)>= 3.1416:     # when angle - overlap is smaller than -pi make angle positive and join dfs in range 
            
            phi_max = phi_edges[i+1] + overlap_phi
            phi_min = ((phi_edges[i] - overlap_phi) + 2*np.pi) 
            pos = df.loc[df.phi > phi_min]
            neg = df.loc[df.phi <= phi_max]
            phi_hits = pd.concat((pos,neg))
            
            
          elif np.round(abs(phi_edges[i+1] + overlap_phi),4)>= 3.1416:  # when angle + overlap is bigger than pi make angle negative and join dfs in range 
            
            phi_min = phi_edges[i] - overlap_phi
            phi_max = ((phi_edges[i+1] + overlap_phi) - 2*np.pi) 
            pos = df.loc[df.phi > phi_min]
            neg = df.loc[df.phi <= phi_max]
            phi_hits = pd.concat((pos,neg))

          else:
            
            phi_min = phi_edges[i] - overlap_phi
            phi_max = phi_edges[i+1] + overlap_phi
            phi_hits = df[(df.phi >= phi_min) & (df.phi < phi_max)]
        

          
        
          # Center these hits on phi=0
          centered_phi = phi_hits.phi - (phi_min + phi_max) / 2
          phi_hits = phi_hits.assign(sec_phi=centered_phi, phi_section=i)
          #loop over sections on phi
          for j in range(len(ita_edges) - 1):
              ita_min, ita_max = ita_edges[j] - overlap_ita, ita_edges[j+1] + overlap_phi
              # Select hits in this ita section
              sec_hits = phi_hits[(df['ita'] >= ita_min) & (df['ita'] < ita_max)]
              hits_sections.append(sec_hits.assign(ita_section=j))
       #create new hits ids 
      for i ,dfs in enumerate(hits_sections):
        dfs = dfs.reset_index(drop = True)
        dfs.new_hit_id = dfs.index
        hits_sections[i] = dfs

      return hits_sections
  return  split_detector_sections(df, phi_edges, ita_edges)


#barrel
def volume_filter(df, volume_list):
  """ 
     Returns only hits in volumes given in volume_list 
  """
  return df.loc[df.volume_id.isin(volume_list)]

#choosen layers
def layer_filter(df, layer_list):
  """ 
     Returns only hits in volumes given in volume_list 
  """
  return df.loc[df.new_layer_id.isin(layer_list)]

In [ ]:

#######################INITIAL FILTERS#####################################
def remove_noise(df):
  return df.loc[df.particle_id!=0]
def weight_filter(df):
  """
     Remove weights = 0
  """
  return df.loc[df.weight!=0]



def db_hit_layer_filter(df):
  """
     Remove double hits in the same layer
  """
  return df.loc[df.duplicated(subset=['new_particle_id', 'layer_id','volume_id'])==False]






In [ ]:
#######################CALCULATION FILTERS#####################################
def p_t_filter(df, min_pt):
  """Removes particles < min_pt"""
  pt_1 = df.loc[df['pt']>min_pt].new_particle_id.unique()
  return df.loc[df['new_particle_id'].isin(pt_1)]
def ita_filter(df, min_ita, max_ita):
  """keeps particles in the range (min_ita,max_ita)"""
  ita_f = df.loc[(df['ita']<min_ita) | (df['ita']>max_ita)].new_particle_id.unique()
  return df.loc[~df['new_particle_id'].isin(ita_f)]

## Node Edge Conections

In [ ]:
#######################MORE THAN 4 FILTERS#####################################
def nhit_filter(df, nhits):
  """
     Remove less than n hits
  """
  #find the amounts of hits per particle
  new_nhits = df.new_particle_id.value_counts().rename_axis('new_particle_id').reset_index(name='new_nhits') 
  #merge the new hhits on new particle id 
  df = df.merge(new_nhits, on='new_particle_id')
  return df.loc[df.new_nhits >= nhits]



In [ ]:
#######################CREATE EDGES#####################################
def calc_dphi(phi1, phi2):
  """Computes phi2-phi1 """
  dphi = phi2 - phi1
   
  return dphi

  
def get_param_distr(df):
  """ create edges and calculates dphi, dz,dr,dita,dR,phi_slope, and z0"""

  dR_list, phi_slope_list,z0_list, hits_pair_list = [], [], [],[]
  #################CREATE EDGES#####################################
  # some particle ids dont have hist in certain layers therefore is not possible to just loop through conecutive layers, 
  # it is neccesary to loop through each p_id to see the set of consecutive layers and conect those layers
  
  for p_id in df.new_particle_id.unique():
    
    temp_df = df.loc[df.new_particle_id==p_id]
    cons_layers = temp_df.new_layer_id.unique()
    # loop trhough layers to generate hits pair(section of track) on consecutive layers 
    for i in range(len(cons_layers)-1):
        
      hits1 = temp_df.loc[temp_df.new_layer_id == cons_layers[i]]
      hits2 = df.loc[df.new_layer_id == cons_layers[i+1]]
      keys = ['event_id', 'new_hit_id','new_layer_id','new_particle_id','r', 'phi', 'z','ita','x','y']
      hit_pairs  = hits1[keys].reset_index(drop = True).merge(hits2[keys].reset_index(drop = True), on='event_id', suffixes=('_1', '_2'))
      
      hit_pairs['y'] = hit_pairs.new_particle_id_1 == hit_pairs.new_particle_id_2


      hits_pair_list.append(hit_pairs)
    
    # concatenate firts 2 hit pairs 
  df_concat = pd.concat([hits_pair_list[0], hits_pair_list[1]], ignore_index=True)
  # concatenate firts the rest hit pairs
  for i in range(len(hits_pair_list) -2):
    df_concat = pd.concat([df_concat, hits_pair_list[i+2]], ignore_index=True)

  #############CALCULATE PARAMTERS#####################################  
  df_concat['dphi'] = calc_dphi(df_concat.phi_1, df_concat.phi_2)
  df_concat['dz'] = df_concat.z_2 - df_concat.z_1
  df_concat['dr'] = df_concat.r_2 - df_concat.r_1
  df_concat['dita'] = df_concat.ita_2 - df_concat.ita_1
  df_concat['dR'] = np.sqrt(df_concat['dita']**2 + df_concat['dphi']**2)
  df_concat['phi_slope'] = df_concat['dphi'] / df_concat['dr']
  df_concat['z0'] = df_concat.z_1 - df_concat.r_1 * df_concat['dz'] / df_concat['dr']
  return df_concat

In [ ]:
def edge_filter(df):
  """reduce amount of edges using calculated parameters"""
  min_z0 =  -320
  max_z0 = 520
  df = df.loc[(df.z0>=min_z0) & (df.z0<=max_z0)]
  
  
  max_dr = 0.5
  df = df.loc[df.dR<=max_dr]
  
  min_phi_slope =  -.0055
  max_phi_slope = .0055
  df =df.loc[(df.phi_slope>=min_phi_slope) & (df.phi_slope<max_phi_slope)] 
  return df


## Inbalanced Data Set

In [ ]:
#############BALANCE DATASET##################################### 
def resample_data(df):
  """Returns balanced dataset"""
  X = df.drop(['y'], axis=1)
  y = df.y
      
  #undersample data 
  rus = RandomUnderSampler(random_state=42)
  X_res, y_res = rus.fit_resample(X, y)
 
  df = X_res
  df['y'] = y_res
  return df.sort_values(by=['new_layer_id_1','z_1'])

##########RE-LABLE HIT ID#####################################
def New_hit_id(df):
  """returns dataset with ordered hit id"""
  df = df.sort_values(by=['new_layer_id_1','z_1'])
  df_hits = pd.DataFrame()
  df_hits['unique_hits'] =  pd.concat([df['new_hit_id_1'], df['new_hit_id_2']]).unique()
  df_hits = df_hits.sort_values('unique_hits').reset_index(drop = True)
  df_hits['new_hit'] = df_hits.index

  newest_id1 = np.zeros(len(df))
  
  for i, id in enumerate(df['new_hit_id_1']):
    newest_id1[i] = df_hits.new_hit.loc[df_hits.unique_hits==id]
  df['hit_id_1'] = newest_id1.astype(int)
  newest_id2 = np.zeros(len(df))
  
  for i, id in enumerate(df['new_hit_id_2']):
    newest_id2[i] = df_hits.new_hit.loc[df_hits.unique_hits==id]
  df['hit_id_2'] = newest_id2.astype(int)
  df = df.sort_values('hit_id_1')

  
  return df

## Data in Graph Format

In [ ]:
def data_to_graph(df,directed=True):
  """Returns graph data into pytorch format"""
  edge_attr = df[['dphi', 'dz', 'dr', 'dita', 'dR','phi_slope', 'z0']]
  edge_index = df[['hit_id_1','hit_id_2']]
  y = df.y
  df1 = pd.DataFrame()
  df2 = pd.DataFrame()
  df1[['hit_id','r', 'phi', 'z', 'ita', 'x','y' ]] = df[['hit_id_1','r_1', 'phi_1', 'z_1', 'ita_1','x_1','y_1' ]]
  df2[['hit_id','r', 'phi', 'z', 'ita', 'x','y' ]] = df[['hit_id_2','r_2', 'phi_2', 'z_2', 'ita_2','x_2','y_2' ]]
  x = pd.concat([df1,df2])
  x = x.drop_duplicates(subset=['hit_id'])
  x = x.sort_values('hit_id')
  x = x[['r', 'phi', 'z', 'ita', 'x','y']]

  if directed:
    edge_attr = torch.from_numpy(edge_attr.to_numpy()).type(torch.FloatTensor)
    edge_index = torch.from_numpy(edge_index.to_numpy())   # this is double 
    y = torch.from_numpy(y.to_numpy()).type(torch.FloatTensor)
    x = torch.from_numpy(x.to_numpy()).type(torch.FloatTensor)    #change to float  x[['r', 'phi', 'z', 'ita', 'x','y']]
    data = Data(x=x, edge_index=edge_index.T,edge_attr=edge_attr,y=y)
  else:

    edge_index_und = torch.stack([torch.cat([edge_index[:,0], edge_index[:,1]], dim=0), torch.cat([edge_index[:,1], edge_index[:,0]], dim=0)], dim = 0)
    edge_attr_und = torch.cat([edge_attr, -1*edge_attr], dim=0)
    y_und = torch.cat([y,y])
    data = Data(x=x,edge_index= edge_index_und,edge_attr = edge_attr_und,y = y_und)#, first_layer_hits = first_layer_hits  )
  return data 

## Run

In [ ]:
##########CONSTANTS#####################################
volume_list = [8,13,17]   #barrel
pt_cut = 1
ita_min = -3
ita_max = 3
nhits = 5
layer_list = [0,1,2,3,4,5,7,8]


def run_and_save(event_start,event_end,path_file, SAVE=True,BALANCE=False):
  """transforms raw data into graph format as outlined in the report and save it"""
  for i in range(event_start,event_end):
    file_idx =0
    #read data 
    hits, particles, truth = read_all_data(1000 + i,1000 + (i+1))  
    df = merge_dfs(hits, particles, truth)
    
    #firts volume filter
    df = volume_filter(df, volume_list)
  
    #fits stage create new_layer_id 
    df = ordered_layer_id(df)
    
    #then return only layers in list 
    df = layer_filter(df, layer_list)

    #calculate the extra parameters
    df = initial_params(df)

    #section detector
    df_sections = sections(df,4,2, overlap_phi = 0.26, overlap_ita = 0.5)

     #loop throug all sections 
  for sec,dfs in enumerate(df_sections):
    #initial filters
    dfs = dfs.pipe(remove_noise).pipe(weight_filter).pipe(db_hit_layer_filter)
    
    #calculation filter 
    dfs = dfs.pipe(initial_params).pipe(p_t_filter,pt_cut).pipe(ita_filter,ita_min, ita_max)
   
   #less than 4 hit filter
    dfs = nhit_filter(dfs, nhits)
  
    #order layers again
    dfs = dfs.pipe(ordered_layer_id)

    # construct graph segments (edges) in the correct range
   
    dfs = dfs.pipe(get_param_distr).pipe(edge_filter)

    # save in different stages for analyisis 
    if SAVE:
      newpath = f'/PATH_TO_SAVE1/'+f'{path_file}/Section_{sec}/'
      if not os.path.exists(newpath):
        os.makedirs(newpath)
      dfs.to_csv(newpath+f'event{file_idx}.csv',index=False)
    
    #resample data 
    if BALANCE:
      #only balance fro train set
      dfs = dfs.pipe(resample_data).pipe(New_hit_id)
    else:
      dfs = dfs.pipe(New_hit_id)

    if SAVE:
      newpath = f'/PATH_TO_SAVE2/'+f'{path_file}/Section_{sec}/'
      if not os.path.exists(newpath):
        os.makedirs(newpath)
      dfs.to_csv(newpath+f'event{file_idx}.csv',index=False)
    
    #data to graph 
    newpath = f'/PATH_TO_FINAL_GRAPH_DATA/'+f'{path_file}/Section_{sec}/'
    if not os.path.exists(newpath):
      os.makedirs(newpath)
    data = data_to_graph(df)
    #save grpah files 
    torch.save(data,newpath + f'data_{file_idx}.pt')
    
    
  file_idx +=1

In [ ]:
#train
run_and_save(0,60, 'Train', SAVE=True,BALANCE=True)
#test
run_and_save(60,90, 'Test', SAVE=True,BALANCE=False)
#val
run_and_save(90,100, 'Val', SAVE=True,BALANCE=False)

In [ ]:
#master model
def create_master_files(start,end,file_name):
  indir = f'/PATH_TO/GRAPH_BALANCED/'
  sections = np.arange(0,8)
  files = []
  for sec in sections:
    print(sec)
    sec_file = f'Section_{sec}/'
    graph_files = np.array(os.listdir(indir + sec_file))
    graph_files = [os.path.join(indir+sec_file,file) for file in graph_files]
    files.append(graph_files)
    newpath = f'/PATH_TO_MASTER/GRAPH_FINAL_{file_name}_MASTER/'
    if not os.path.exists(newpath):
      os.makedirs(newpath)
  file_idx=0
  for i,f in enumerate(files[0][start:end]):
    for j in range(8):
      
      
      df = pd.read_csv(files[j][i]) 
      data = data_to_graph(df)
      torch.save(data,newpath + f'data_{file_idx}.pt')
      file_idx+=1

In [ ]:
create_master_files(0,60,'TEST')
create_master_files(60,90,'TRAIN')
create_master_files(90,100,'VAL')